In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
df = pd.read_csv("../input/product-position/upper.csv", header=None)
df.head()

In [4]:
df.shape

In [5]:
df.groupby(0)[0].count()

In [6]:
df_sel = df[(df[0]==0) | (df[0]==8) | (df[0]==9)]
df_sel.shape

In [11]:
Xo = df_sel.drop([0],axis=1).values
yo = df_sel[0].values

In [12]:
Xo.shape

In [13]:
yo.shape

In [14]:
np.unique(yo)

In [15]:
yo[yo==8] = 1
yo[yo==9] = 2
np.unique(yo)

In [16]:
[sum(yo == item) for item in np.unique(yo)]

In [17]:
from imblearn.under_sampling import RandomUnderSampler
sm = RandomUnderSampler(random_state=1)
X_res, y_res = sm.fit_resample(Xo,yo)

In [18]:
from sklearn.model_selection import train_test_split #มีข้อมูล 2 set
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=1)

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm =  scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [20]:
import time
import sys
from time import sleep
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier

t0 = time.time()
pr = {
    'n_estimators' : [10,50,100,200,300],
    'min_samples_leaf' : [1,2,4,8,16,32],
}

for i in tqdm(pr):
    best_clf = GridSearchCV(ExtraTreesClassifier(random_state=0),pr,cv=10) 
    best_clf.fit(X_train_norm, y_train)
print ("Training time: " + str(round(time.time()-t0,2)) + " s")

In [21]:
yp = best_clf.predict(X_test_norm)
acc_test = sum(yp == y_test)/len(y_test)
print("Best score is " + str(best_clf.best_score_*100) +('\n') +"Best param is " + str(best_clf.best_params_))
print("Test Training Acc. : " + str(acc_test*100))

In [22]:
df = pd.read_csv("../input/product-position/upper_test.csv", header=None)
df.head()

In [23]:
df_sel = df[(df[0]==0) | (df[0]==8) | (df[0]==9)]
df_sel.shape

In [24]:
Xb = df_sel.drop([0],axis=1).values
yb = df_sel[0].values

In [25]:
yb[yb==8] = 1
yb[yb==9] = 2
np.unique(yb)

In [27]:
X_blind_norm = scaler.transform(Xb)

In [28]:
yp = best_clf.predict(X_blind_norm)
acc_test = sum(yp == yb)/len(yb)
print("Best score is " + str(best_clf.best_score_*100) 
print("Best param is " + str(best_clf.best_params_))
print("Test Training Acc. : " + str(acc_test*100))